Clase 11

# <center>spaCy</center>

---

[spaCy](https://spacy.io/) es una librería open-source diseñada para realizar tareas de NLP.
Su código, disponible en [GitHub](https://github.com/explosion/spaCy), se encuentra escrito en Python y Cython. Este último permite enriquecer la escritura de Python con propiedades de los lenguajes C y C++, haciéndolo más eficiente.

**Guía de clase:**

- [1. Importar la librería](#1.-Importar-la-librería)
- [2. Configurar un pipeline](#2.-Configurar-un-pipeline)
- [3. Doc](#3.-Doc)
- [4. Tokenización](#4.-Tokenización)
- [5. Etiquetado](#5.-Etiquetado)
- [6. Parseo de dependencias](#6.-Parseo-de-dependencias)
- [7. Segmentación de oraciones](#7.-Segmentación-de-oraciones)
- [8. NER](#8.-NER)
- [9. Matcheo-mediante-reglas](#9.-Matcheo-mediante-reglas)

## 1. Importar la librería

In [1]:
import sys
sys.path.append("/home/dipa/proyectos/2019_cursoNLP/lib/python3.6/site-packages")
import spacy
from spacy import displacy

SpaCy cuenta con una serie de modelos pre-entrenados para distintas lenguas (inglés, alemán, español, francés, portugués, italiano, holandés y griego). Para poder usarlos, es necesarios instalarlos luego de instalar la librería, del mismo modo que instalamos cualquier paquete de Python.

Asimismo, una vez importada la librería, debe instanciarse con el entrenamiento deseado. De él tomará los distintos componentes para los análisis lingüísticos.

In [2]:
# cargamos un entrenamiento en español

nlp = spacy.load("es_core_news_sm")

Actualmente, SpaCy cuenta con dos modelos pre-entrenados para el español:

- es_core_news_sm: modelo multipropósito entrenado con textos de medios y redes sociales. Contiene componentes para realizar tokenización, POS tagging, parsers de dependencia y reconocer entidades (PER, LOC, ORG, MISC)

- es_core_news_md: modelo multipropósito que, además de contener los componentes del modelo anterior, presenta un tamaño mayor e incluye vectores de palabras

Ambos modelos toman como fuente textos extraídos de AnCora, Wikipedia y WikiNER Corpus.

## 2. Configurar un pipeline

![alt text](./pictures/pipeline.svg)

El atributo *pipe_names* nos permite acceder a los nombre de los componentes que se encuentran incluidos en el modelo. Estos se muestran en orden de ejecución.

In [3]:
nlp.pipe_names

['tagger', 'parser', 'ner']

Si no quisiéramos utilizar todos los componentes incluidos (ya sea porque no los necesitamos o porque queremos cambiar el componente por defecto por uno hecho por nosotros mismos), podemos deshabilitar aquellos que queramos excluir utilizando al momento de cargar el idioma, o bien deshabilitándolos luego con el método *disable_pipes*.

In [8]:
nlp_parser = spacy.load("es", disable=["ner", "tagger"])


nlp_parser.pipe_names

OSError: [E050] Can't find model 'es'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [9]:
disabled = nlp.disable_pipes('tagger', 'parser')
nlp.begin_training()

In [10]:
disabled

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f2f6af59ef0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f2f6af45168>)]

In [11]:
nlp.pipe_names

['ner']

Y también podemos restaurar los valores por default.

In [12]:
disabled.restore()

In [13]:
nlp.pipe_names

['tagger', 'parser', 'ner']

## 3. Doc

SpaCy ya viene con información lingüística y anotaciones que podemos utilizar para enriquecer nuestros textos crudos. Para ello, debemos construir un objeto ```Doc```, el cual consite en una secuencia de objetos ```Token``` con distintas anotaciones según cuál sea el modelo pre-entrenado que estemos utilizando y cómo hayamos configurado nuestro pipeline.

In [4]:
with open('bestiario/casa_tomada.txt','r') as f:
    casa_tomada = f.read()

In [5]:
print(casa_tomada[:1000])

CASA TOMADA

 	Nos gustaba la casa porque aparte de espaciosa y antigua (hoy que las casas antiguas sucumben a la más ventajosa liquidación de sus materiales) guardaba los recuerdos de nuestros bisabuelos, el abuelo paterno, nuestros padres y toda la infancia.
 	Nos habituamos Irene y yo a persistir solos en ella, lo que era una locura pues en esa casa podían vivir ocho personas sin estorbarse. Hacíamos la limpieza por la mañana, levantándonos a las siete, y a eso de las once yo le dejaba a Irene las últimas habitaciones por repasar y me iba a la cocina. Almorzábamos a mediodía, siempre puntuales; ya no quedaba nada por hacer fuera de unos pocos platos sucios. Nos resultaba grato almorzar pensando en la casa profunda y silenciosa y cómo nos bastábamos para mantenerla limpia. A veces llegamos a creer que era ella la que no nos dejó casarnos. Irene rechazó dos pretendientes sin mayor motivo, a mí se me murió María Esther antes que llegáramos a comprometernos. Entramos en los cuarenta año

In [6]:
doc = nlp(casa_tomada)

## 4. Tokenización

In [7]:
import os
import re

Cantidad de tokens en el texto:

In [8]:
doc.__len__()

2212

In [9]:
for token in doc[:20]:
    print(token.i, token.text)

0 CASA
1 TOMADA
2 

 	
3 Nos
4 gustaba
5 la
6 casa
7 porque
8 aparte
9 de
10 espaciosa
11 y
12 antigua
13 (
14 hoy
15 que
16 las
17 casas
18 antiguas
19 sucumben


Tokenizador de SpaCy:

1. El texto crudo se segmenta por espacios, como cuando usamos text.split(' ')
2. El tokenizador recorre el texto de izquierda a derecha y, para cada substring o posible palabra encontrada, realiza dos verificaciones:

    2.1. Evalúa si la posible palabra forma parte de alguna excepción y que entonces deba seguir alguna regla particular
    
    2.2 Busca signos especiales como comas, paréntesis o comillas que puedan ser abiertos, cerrados o estar entre las palabras y deban separarse de ellas.
    
    Si encuentra alguno de estos casos, el tokenizador separa la substring considerando estos caracteres y sigue recorriendo el texto.

Un posible caso de excepción:

In [10]:
excepcion = "bieniusté"

doc_ex = nlp(excepcion)

for token in doc_ex:
    print(token.text)

bieniusté


**Cómo agregar reglas que contemplen excepciones:**

In [11]:
special_case = [{'ORTH': u"bien", 'LEMMA': u"bien", 'POS': u"ADJ"},{'ORTH': 'i','LEMMA':'y','POS':'CCONJ'},
               {'ORTH':'usté','LEMMA':'usted','POS':'PRON'}]

nlp.tokenizer.add_special_case(u"bieniusté", special_case)

# vemos la nueva tokenización
print([w.text for w in nlp(u"bieniusté")]) 

# vemos la lematización de los tokens
print([w.lemma_ for w in nlp(u"bieniusté")]) 

['bien', 'i', 'usté']
['bien', 'y', 'usted']


**Cómo armar un tokenizador propio:**

El tokenizador que incorporemos a nuestro pipeline debe tomar como input un texto y devolver un objeto Doc que pueda integrarse con el resto de los componentes.

In [12]:
from nltk import word_tokenize
from spacy.tokens import Doc

class NLTKTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        # Aquí le indicamos al tokenizador qué queremos que haga cuando lo llamamos
        words = word_tokenize(text)
        # Todos los tokens poseen un espacio que le sigue
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

Cambiamos el tokenizador por defecto por el de NLTK que importamos y vemos qué pasa:

In [14]:
# instanciamos el tokenizador con un vocabulario
#nlp_parser.tokenizer = NLTKTokenizer(nlp.vocab)

nlp_parser = NLTKTokenizer(nlp.vocab)

doc_excepcion = nlp_parser(excepcion)

ex_tokens = [token.text for token in nlp_parser(excepcion)]
ex_tokens

['bieniusté']

También podría ser que nos interesase conocer las palabras que se encuentran alredeor los tokens. El método ```nbor``` nos permite acceder a esta información:

In [15]:
n_words = 4

for token in doc:
    if token.text == "Irene":
        circundantes = [token.nbor(i) for i in range(1,n_words)]
        print(token.i, token.text)
        print(circundantes)

50 Irene
[y, yo, a]
97 Irene
[las, últimas, habitaciones]
164 Irene
[rechazó, dos, pretendientes]
265 Irene
[era, una, chica]
322 Irene
[no, era, así]
395 Irene
[tenía, fe, en]
461 Irene
[,, porque, yo]
474 Irene
[sin, el, tejido]
535 Irene
[qué, pensaba, hacer]
564 Irene
[solamente, la, entretenía]
833 Irene
[y, yo, vivíamos]
967 Irene
[estaba, tejiendo, en]
1136 Irene
[:, 
 	, —Tuve]
1257 Irene
[extrañaba, unas, carpetas]
1282 Irene
[pensó, en, una]
1375 Irene
[se, acostumbró, a]
1406 Irene
[cocinaría, platos, para]
1443 Irene
[y, las, fuentes]
1452 Irene
[estaba, contenta, porque]
1518 Irene
[que, era, más]
1525 Irene
[decía, :, 
 	]
1597 Irene
[soñaba, en, alta]
1631 Irene
[decía, que, mis]
1763 Irene
[cantaba, canciones, de]
1835 Irene
[empezaba, a, soñar]
1871 Irene
[que, iba, hasta]
1921 Irene
[le, llamó, la]
1992 Irene
[y, la, hice]
2047 Irene
[., El, tejido]
2150 Irene
[(, yo, creo]


# 5. Etiquetado

Lugo de tokenizar, el pipeline de SpaCy etiquetar y parsea el Doc.Para ello, usa un modelo estadísico para predecir cuál es el tag o etiqueta más adecuado para el contexto.

La información que podemos extraer de estas etiquetas es la siguiente:

- **Text:** Texto de la palabra original.
- **Lemma:** Forma base de la palabra
- **POS:** POS tag simple..
- **Tag:** POS tag detallado.
- **Dep:** Dependencia sintáctica.
- **Shape:** Forma de la palabra - puntuación, dígitos, mayúsculas.
- **is alpha:** ¿El token es alfabético?
- **is stop:** ¿El token es una stopword?

In [16]:
import pandas as pd

tokens_info = {'text':[],
              'lemma':[],
              'pos':[],
              'tag':[],
              'dep':[],
              'shape':[],
              'is_alpha':[],
              'is_stop':[]}
    

for token in doc:
    tokens_info['text'].append(token.text)
    tokens_info['lemma'].append(token.lemma_)
    tokens_info['pos'].append(token.pos_)
    tokens_info['tag'].append(token.tag_)
    tokens_info['dep'].append(token.dep_)
    tokens_info['shape'].append(token.shape_)
    tokens_info['is_alpha'].append(token.is_alpha)
    tokens_info['is_stop'].append(token.is_stop)
    
df = pd.DataFrame(tokens_info)
df.head()

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop
0,CASA,CASA,NOUN,NOUN__Gender=Fem|Number=Sing,nsubj,XXXX,True,False
1,TOMADA,TOMADA,PROPN,PROPN___,amod,XXXX,True,False
2,\n\n \t,\n\n \t,SPACE,_SP,,\n\n \t,False,False
3,Nos,Nos,PRON,PRON__Number=Plur|Person=1|PronType=Prs,obj,Xxx,True,True
4,gustaba,gustar,VERB,VERB__Mood=Ind|Number=Sing|Person=3|Tense=Imp|...,ROOT,xxxx,True,False


Si tenemos dudas sobre las etiquetas, podemos obtener un poco más de información sobre ellas de la siguiente manera:

In [17]:
spacy.explain('PROPN')

'proper noun'

**Cómo funciona el sistema de etiquetado:**

1. El tokenizador consulta si existen excpciones de tokenización que impliquen que una secuencia de caracteres se mapee con más de un token, y le asigna una etiqueta POS a cada token encontrado.

2. Luego, se le asigna a cada token una etiqueta POS extendida con la información morfológica (```tag_```).

3. Si algún token no recibió una etiqueta POS en el proceso anterior, se utiliza una tabla de mapeo para estableces sus etiquetas.

4. Por último, una regla determinísitca establece cuál es el lema del token en cuestión, considerando tanto la etiqueta POS como la información morfológica encontrada. En el caso del inglés, también se pueden utilizar archivos de excepciones provistos por [WordNet](https://wordnet.princeton.edu/).

**Links útiles:**

[Universal POS tags](https://universaldependencies.org/u/pos/)

[Mapeo de tags para español](https://github.com/explosion/spaCy/blob/master/spacy/lang/es/tag_map.py)

## 6. Parseo de dependencias

In [18]:
sentences = [sent for sent in doc.sents]
sentences[6]

Irene rechazó dos pretendientes sin mayor motivo, a mí se me murió María Esther antes que llegáramos a comprometernos.

In [19]:
options = {"compact": True,"bg": "#09a3d5", "distances":1,
           "color": "white", "font": "Source Sans Pro"}

displacy.render(sentences[6], style="dep", jupyter=True, options=options)

# para ver en otra página
# displacy.serve(sentences[6], style="dep", options=options)

**Noun chunks**

Un noun chunk es una frase nominal base (NP en inglés) que no contiene otra frase nominal incrustada (i.e. no contiene coordinación nominal, frases preposicionales ni cláusulas relativas).

SpaCy contiene algunos métodos que nos permiten acceder a la información de estas porciones de texto:

- Text: texto original
- Root.text: texto original de la palabra que conecta el chunk con el resto de la oracion (núcleo del chunk)
- Root.dep_: relación de dependencia que conecta el núcleo del chunk con el núcleo de laoración donde se inserta
- Root_head text: núcleo de la oración donde se encuentra el chunk

In [20]:
print(sentences[6].text, end='\n\n')
sent6 = nlp(sentences[6].text)

print('{0:30} {1:15} {2:15} {3}'.format('text','root','dep','head'), end='\n\n')

for chunk in sent6.noun_chunks:
    print('{0:30} {1:15} {2:15} {3}'.format(chunk.text, 
                                            chunk.root.text, 
                                            chunk.root.dep_, 
                                            chunk.root.head.text))

Irene rechazó dos pretendientes sin mayor motivo, a mí se me murió María Esther antes que llegáramos a comprometernos.

text                           root            dep             head

Irene                          Irene           nsubj           rechazó
pretendientes                  pretendientes   obj             rechazó
motivo                         motivo          obl             rechazó
mí                             mí              obj             murió
se                             se              obj             murió
me                             me              obj             murió
María Esther                   María           nsubj           murió


In [ ]:
print('{0:15} {1:15} {2:15} {3:15} {4}'.format('text','dep','head text','head pos', 'children'), end='\n\n')

for token in sent6:
    print('{0:15} {1:15} {2:15} {3:15} {4}'.format(token.text, 
                                                   token.dep_, 
                                                   token.head.text, 
                                                   token.head.pos_,
                                                   [child for child in token.children]))

Así como podemos ver los nodos hijos de un token, también podemos ver sus  nodos padres o ancestrales:

In [ ]:
for token in sent6:
    print('{0:15} {1}'.format(token.text,
                           [ant for ant in token.ancestors]))

**Links útiles:**

[Online demo para visualizar parseos de dependencias](https://explosion.ai/demos/displacy)

## 7. Segmentación de oraciones

SpaCy utiliza el parser de dependencias para determinar los límites de las oraciones en lugar de hacerlo mediante reglas. Esto hace que los límites encontrados dependan del modelo estadístico que se haya entrenado previamente. Sin embargo, también es posible añadir patrones de delimitación o cambiar el segmentador por otro.

In [21]:
i = 1
for sent in doc.sents:
    print('ORACIÓN',i)
    print(sent.text)
    print('-----')
    i += 1

ORACIÓN 1
CASA TOMADA

 	Nos gustaba la casa porque aparte de espaciosa y antigua (hoy que las casas antiguas sucumben a la más ventajosa liquidación de sus materiales) guardaba los recuerdos de nuestros bisabuelos, el abuelo paterno, nuestros padres y toda la infancia.
 	
-----
ORACIÓN 2
Nos habituamos Irene y yo a persistir solos en ella, lo que era una locura pues en esa casa podían vivir ocho personas sin estorbarse.
-----
ORACIÓN 3
Hacíamos la limpieza por la mañana, levantándonos a las siete, y a eso de las once yo le dejaba a Irene las últimas habitaciones por repasar y me iba a la cocina.
-----
ORACIÓN 4
Almorzábamos a mediodía, siempre puntuales; ya no quedaba nada por hacer fuera de unos pocos platos sucios.
-----
ORACIÓN 5
Nos resultaba grato almorzar pensando en la casa profunda y silenciosa y cómo nos bastábamos para mantenerla limpia.
-----
ORACIÓN 6
A veces llegamos a creer que era ella la que no nos dejó casarnos.
-----
ORACIÓN 7
Irene rechazó dos pretendientes sin mayo

Si chequeamos el árbol de dependencias, efectivamente vemos que el parser hace depender el título de la primera oración.

In [22]:
sent1 = nlp(sentences[0].text)
options={"distances":1}
displacy.render(sent1, style="dep", options=options, jupyter=True)

Para solucionar esto, SpaCy nos ofrece dos estrategias:

- Añadir un componente llamado ```sentencizer``` que separa las oraciones por signos de puntuación tales como el punto (.), el signo de pregunta (?) o el signo de exclamación (!)
- Armar nuestro propio segmentador de oraciones basado en reglas y agregarlo al pipeline

Para armar un segmentador propio, debemos definir una función que establezca cuál será nuestro separador de oraciones.

In [23]:
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if re.search('\n\n',token.text):
            doc[token.i+1].is_sent_start = True
    return doc

Luego, agregamos nuestra función al pipeline y volvemos a cargar el documento para que realice la segmentación nuevamente.

In [24]:
nlp.add_pipe(set_custom_boundaries, name="sent_segmenter", before="parser")
nlp.pipe_names

['tagger', 'sent_segmenter', 'parser', 'ner']

In [25]:
import re

doc = nlp(casa_tomada)
sentences_good = list()

i = 1
for sent in doc.sents:
    print('ORACIÓN',i)
    print(sent.text)
    sentences_good.append(sent.text.strip())
    print('-----')
    i += 1

ORACIÓN 1
CASA TOMADA

 	
-----
ORACIÓN 2
Nos gustaba la casa porque aparte de espaciosa y antigua (hoy que las casas antiguas sucumben a la más ventajosa liquidación de sus materiales) guardaba los recuerdos de nuestros bisabuelos, el abuelo paterno, nuestros padres y toda la infancia.
 	
-----
ORACIÓN 3
Nos habituamos Irene y yo a persistir solos en ella, lo que era una locura pues en esa casa podían vivir ocho personas sin estorbarse.
-----
ORACIÓN 4
Hacíamos la limpieza por la mañana, levantándonos a las siete, y a eso de las once yo le dejaba a Irene las últimas habitaciones por repasar y me iba a la cocina.
-----
ORACIÓN 5
Almorzábamos a mediodía, siempre puntuales; ya no quedaba nada por hacer fuera de unos pocos platos sucios.
-----
ORACIÓN 6
Nos resultaba grato almorzar pensando en la casa profunda y silenciosa y cómo nos bastábamos para mantenerla limpia.
-----
ORACIÓN 7
A veces llegamos a creer que era ella la que no nos dejó casarnos.
-----
ORACIÓN 8
Irene rechazó dos prete

In [26]:
sentences_good[:3]

['CASA TOMADA',
 'Nos gustaba la casa porque aparte de espaciosa y antigua (hoy que las casas antiguas sucumben a la más ventajosa liquidación de sus materiales) guardaba los recuerdos de nuestros bisabuelos, el abuelo paterno, nuestros padres y toda la infancia.',
 'Nos habituamos Irene y yo a persistir solos en ella, lo que era una locura pues en esa casa podían vivir ocho personas sin estorbarse.']

## 8. NER

Un NER (Named Entity Recognition) es un sistema que no solo nos permite encontrar o reconocer entidades sino, además, saber de qué tipo son.

Ahora bien, ¿qué es una entidad? ¿Cuántos tipos de entidades hay o puede haber? ¿Para qué puede servirnos reconocer entidades?

El NER de SpaCy nos permite reconocer los siguientes atributos de una entidad:

- Text: texto original de la entidad
- Start: índice de inicio de la entidad en el Doc
- End: índice de finalización de la entidad en el Doc
- LabeL: etiqueta de la entidad (tipo de entidad)

In [27]:
print(sentences_good[7])
sent7 = nlp(sentences_good[7])

for ent in sent7.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Irene rechazó dos pretendientes sin mayor motivo, a mí se me murió María Esther antes que llegáramos a comprometernos.
María Esther 67 79 ORG


In [28]:
print('{0:50} {1:10} {2:2} {3}'.format('text','start','end','label'),end='\n\n')

for ent in doc.ents:
    print('{0:50} {1:5} {2:8} {3}'.format(ent.text, ent.start_char, ent.end_char, ent.label_))

text                                               start      end label

CASA TOMADA                                            0       11 ORG
Nos gustaba la casa                                   15       34 MISC
Hacíamos                                             398      406 MISC
Irene                                                496      501 PER
Almorzábamos a mediodía                              561      584 MISC
Nos resultaba grato almorzar                         669      697 MISC
A veces llegamos a creer                             786      810 MISC
Irene                                                853      858 PER
María Esther                                         920      932 PER
Entramos                                             972      980 LOC
Nos moriríamos allí algún día                       1174     1203 MISC
Aparte                                              1475     1481 MISC
No sé por qué tejía tanto                           1571     1596 MISC
Irene    

In [29]:
for sent in sentences_good[1:]:
    print(sent, end='\n\n')
    s = nlp(sent)
    #print('{0:50} {1:10} {2:2} {3}'.format('text','start','end','label'),end='\n\n')
    for ent in s.ents:
        #print('{0:50} {1:5} {2:8} {3}'.format(ent.text, ent.start_char, ent.end_char, ent.label_))
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
    print('-------------')

Nos gustaba la casa porque aparte de espaciosa y antigua (hoy que las casas antiguas sucumben a la más ventajosa liquidación de sus materiales) guardaba los recuerdos de nuestros bisabuelos, el abuelo paterno, nuestros padres y toda la infancia.

-------------
Nos habituamos Irene y yo a persistir solos en ella, lo que era una locura pues en esa casa podían vivir ocho personas sin estorbarse.

Irene 15 20 PER
-------------
Hacíamos la limpieza por la mañana, levantándonos a las siete, y a eso de las once yo le dejaba a Irene las últimas habitaciones por repasar y me iba a la cocina.

Irene 98 103 PER
-------------
Almorzábamos a mediodía, siempre puntuales; ya no quedaba nada por hacer fuera de unos pocos platos sucios.

-------------
Nos resultaba grato almorzar pensando en la casa profunda y silenciosa y cómo nos bastábamos para mantenerla limpia.

-------------
A veces llegamos a creer que era ella la que no nos dejó casarnos.

-------------
Irene rechazó dos pretendientes sin mayor

¿No 0 3 MISC
-------------
Un rato después era yo el que le ponía ante los ojos un cuadradito de papel para que viese el mérito de algún sello de Eupen y Malmédy.

Eupen 119 124 LOC
Malmédy 127 134 LOC
-------------
Estábamos bien, y poco a poco empezábamos a no pensar.

-------------
Se puede vivir sin pensar.

-------------
(Cuando Irene soñaba en alta voz yo me desvelaba en seguida.

Irene 8 13 PER
-------------
Nunca pude habituarme a esa voz de estatua o papagayo, voz que viene de los sueños y no de la garganta.

-------------
Irene decía que mis sueños consistían en grandes sacudones que a veces hacían caer el cobertor.

-------------
Nuestros dormitorios tenían el living de por medio, pero de noche se escuchaba cualquier cosa en la casa.

-------------
Nos oíamos respirar, toser, presentíamos el ademán que conduce a la llave del velador, los mutuos y frecuentes insomnios.

-------------
Aparte de eso todo estaba callado en la casa.

-------------
De día eran los rumores doméstic

In [30]:
print('{0:50} {1:10} {2:2} {3}'.format('text','start','end','label'),end='\n\n')

for sent in sentences_good[1:]:
    s = nlp(sent)
    for ent in s.ents:
        print('{0:50} {1:5} {2:8} {3}'.format(ent.text.strip(), ent.start_char, ent.end_char, ent.label_))

text                                               start      end label

Irene                                                 15       20 PER
Irene                                                 98      103 PER
María Esther                                          67       79 ORG
Irene                                                  0        5 PER
Irene                                                  0        5 PER
Irene                                                 47       52 PER
la Argentina                                          37       49 LOC
Irene                                                 59       64 PER
Irene                                                 30       35 PER
Irene                                                 85       90 PER
Irene                                                  7       12 PER
Cómo                                                   0        4 PER
Rodríguez Peña                                       130      144 PER
Irene            

**Visualización:**

In [31]:
displacy.render(sent7, style="ent", jupyter=True)

In [32]:
colors = {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
options = {"ents": ["ORG"], "colors": colors}
displacy.render(sent7, style="ent", options=options, jupyter=True)

Cómo agregar etiquetas sin entrenamiento:

In [33]:
from spacy.tokens import Span

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in sent7.ents]
print('ANTES', ents)


PER = doc.vocab.strings[u"PER"]

per_ent = [Span(sent7, 0, 1, label=PER),Span(sent7, 13, 15, label=PER)]

sent7.ents = per_ent

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in sent7.ents]
print('After', ents)

ANTES [('María Esther', 67, 79, 'ORG')]
After [('Irene', 0, 5, 'PER'), ('María Esther', 67, 79, 'PER')]


Otra forma de accedet al tipo de oración es utilizando el atributo ent_type_ del token:

In [34]:
print(doc[97])
print(doc[97].ent_type_)

Irene
PER


**Links útiles:**

[Anotaciones del NER de Spacy](https://spacy.io/api/annotation#named-entities)

## 9.Matcheo mediante reglas

Rules-base matcher es un componente de SpaCy que posibilita generar reglas con las que contrastar el texto y extraer los fragmentos que se adecúen a ellas. Lo interesante de este componente es que permite combinar expresiones regulares con las anotaciones lingüísticas de los modelos pre-entrenados.

In [40]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

patternSV = [{"ORTH": "Irene", "DEP":'nsubj'},{"TEXT":{"REGEX":"\w+"},"OP":"*"},{"POS": "VERB", "DEP": "ROOT"},]
patternVS = [{"POS": "VERB", "DEP": "ROOT"},{"TEXT":{"REGEX":"\w+"},"OP":"*"},{"ORTH": "Irene", "DEP":'nsubj'}]
patternCOP = [{"ORTH": "Irene", "DEP":'nsubj'},{"TEXT":{"REGEX":"\w+"},"OP":"*"},{"LEMMA":"ser"}]

matcher.add("IreneSV", None, patternSV)
matcher.add("IreneVS", None, patternVS)
matcher.add("IreneCOP", None, patternCOP)

matched_sents = list()

for sent in sentences_good[1:]:
    s = nlp(sent)
    matches = matcher(s)
    match_ents = list()
    if len(matches) > 0:
        for match in matches:
            match_id, start, end = match
            span = s[start:end]
            info_match = {
                "start": span.start_char,
                "end": span.end_char,
                "label": "MATCH"
            }
            match_ents.append(info_match)            
        matched_sents.append({"text": sent, "ents": match_ents})

        
colors = {"MATCH": "linear-gradient(90deg, #b7deed, #49a5bf)"}
options = {"ents": ["MATCH"], "colors": colors}
displacy.render(matched_sents, style="ent", options=options, jupyter=True, manual=True)

AttributeError: 'dict' object has no attribute 'ents'

[Matcheo mediante reglas](https://explosion.ai/demos/matcher)

[Atributos disponibles para tokens](https://spacy.io/api/token#attributes)